In [1]:
import sys
sys.path.append('..')

In [2]:
import os
import csv

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import TensorDataset
import pandas as pd

from defences.util import score
from models.cifar10 import Resnet, Vgg
from models.torch_util import predict_numpy

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
PATH_OUTPUTS = os.path.join('..', 'csv')
if not os.path.exists(PATH_OUTPUTS):
    os.makedirs(PATH_OUTPUTS)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('device:', device)

device: cuda


## Banknote

In [6]:
def get_dataframe(df, model, data, nmodel, nattack, ndefence):
    path_result = os.path.join('..', 'results', '{}_{}_{}_{}.pt'.format(data, nmodel, nattack, ndefence))
    result = torch.load(path_result)
    X_test = result['X_test']
    y_test = result['y_test']
    labels_test = result['labels_test']
    res_test = result['res_test']

    attack_eps = attack.split('_')

    idx_tp = np.where(labels_test == 1)[0]
    pred = predict_numpy(model, X_test[idx_tp], device=device)
    attack_score = np.sum(pred == y_test[idx_tp]) / len(idx_tp)
    my_score = score(res_test[idx_tp], y_test[idx_tp], pred, labels_test[idx_tp])

    block_rate = np.sum(res_test[idx_tp] == labels_test[idx_tp]) / len(idx_tp)

    idx_fp = np.where(labels_test == 0)[0]
    fpr = np.sum(res_test[idx_fp]) / len(idx_fp)
    if ndefence == 'rc':
        block_rate = -1
        fpr = np.sum(res_test[idx_fp] != y_test[idx_fp]) / len(idx_fp)

    df = df.append({
        'Attack': attack_eps[0], 
        'Epsilon': attack_eps[1], 
        'Defence': ndefence,
        'Accuracy after attack': attack_score * 100,
        'Score': my_score * 100,
        'Block Rate': block_rate * 100, 
        'False Positive Rate': fpr * 100
    }, ignore_index=True)
    return df

In [7]:

name_data = 'cifar10'
name_model = 'resnet'
name_attacks = [
    'apgd_0.031',
    'apgd_0.3',
    'apgd_0.6',
    'apgd_1.0',
    'apgd_1.5',
    'apgd1_0.3',
    'apgd1_0.6',
    'apgd1_1.0',
    'apgd1_1.5',
    'apgd1_2.0',
    'apgd1_3.0',
    'apgd1_5.0',
    'apgd2_0.3',
    'apgd2_0.6',
    'apgd2_1.0',
    'apgd2_1.5',
    'apgd2_2.0',
    'apgd2_3.0',
    'apgd2_5.0',
    'cw2_0.0',
    'cw2_10.0',
    'cw2_5.0',
    'cwinf_0.0',
    'cwinf_5.0',
    'deepfool_1e-06',
    'fgsm_0.031',
    'fgsm_0.3',
    'fgsm_0.6',
    'fgsm_1.0',
    'fgsm_1.5',
    'line_0.0',
    'line_0.5',
    'line_1.0',
    'watermark_0.1',
    'watermark_0.3',
    'watermark_0.6',
    'watermark_1.0',
]
name_defences = [
    'baard_2stage',
    'baard_3stage',
    'fs',
    'lid',
    'rc',
    'magnet',
]

In [8]:
COLUMNS = ['Attack', 'Epsilon', 'Defence', 'Accuracy after attack', 'Score', 'Block Rate', 'False Positive Rate']

In [9]:
NAME_MODEL_FILE = 'cifar10_resnet_200.pt'
model = Resnet(use_prob=True)
path_model = os.path.join('..', 'results', NAME_MODEL_FILE)
model.load_state_dict(torch.load(path_model))
model = model.to(device)

In [10]:
df = pd.DataFrame(columns=COLUMNS)

for attack in name_attacks:
    for defence in name_defences:
        try:
            df = get_dataframe(df, model, name_data, name_model, attack, defence)
        except FileNotFoundError:
            continue    

In [11]:
df.to_csv(os.path.join(PATH_OUTPUTS, '{}_{}.csv'.format(name_data, name_model)))

## VGG

In [12]:
name_model = 'vgg'
NAME_MODEL_FILE = 'cifar10_vgg_200.pt'

model = Vgg(use_prob=True)
path_model = os.path.join('..', 'results', NAME_MODEL_FILE)
model.load_state_dict(torch.load(path_model))
model = model.to(device)

In [13]:
df = pd.DataFrame(columns=COLUMNS)

for attack in name_attacks:
    for defence in name_defences:
        try:
            df = get_dataframe(df, model, name_data, name_model, attack, defence)
        except FileNotFoundError:
            continue   

In [14]:
df.to_csv(os.path.join(PATH_OUTPUTS, '{}_{}.csv'.format(name_data, name_model)))